<a href="https://colab.research.google.com/github/thehungtran/FFDnet_clone/blob/main/ffdnet_projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Imports requirements
Allow GPU by *Execution - Modify execution type - GPU*

In [1]:
from google.colab import drive
import os, glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
from skimage.metrics import structural_similarity as ssim
import cv2


In [5]:
! git clone https://github.com/thehungtran/FFDnet_clone.git

Cloning into 'FFDnet_clone'...
remote: Enumerating objects: 489, done.
remote: Counting objects: 100% (489/489), done.
remote: Compressing objects: 100% (487/487), done.
remote: Total 489 (delta 20), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (489/489), 16.03 MiB | 28.49 MiB/s, done.
Resolving deltas: 100% (20/20), done.


# 1. Run a test

In [7]:
! python /content/FFDnet_clone/test_ffdnet_ipol.py \
	--input /content/FFDnet_clone/test/test001.png \
	--noise_sigma 25 \
	--add_noise True


### Testing FFDNet model ###
> Parameters:
	add_noise: True
	input: /content/FFDnet_clone/test/test001.png
	suffix: 
	noise_sigma: 0.09803921568627451
	dont_save_results: False
	no_gpu: False
	cuda: True


rgb: False
im shape: (481, 321)
Loading model ...

Traceback (most recent call last):
  File "/content/FFDnet_clone/test_ffdnet_ipol.py", line 193, in <module>
    test_ffdnet(**vars(argspar))
  File "/content/FFDnet_clone/test_ffdnet_ipol.py", line 82, in test_ffdnet
    state_dict = torch.load(model_fn)
                 ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/serialization.py", line 1425, in load
    with _open_file_like(f, "rb") as opened_file:
         ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/serialization.py", line 751, in _open_file_like
    return _open_file(name_or_buffer, mode)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/serialization.py", line 732,

# 2. Training

## 1. Prepare dataset

In [8]:
! python /content/FFDnet_clone/prepare_patches.py \
	--trainset_dir /content/FFDnet_clone/training/ \
	--valset_dir /content/FFDnet_clone/test/ \
  --gray \
  --max_number_patches 128 # this can be suppressed, I just set it to run a quick training



### Building databases ###
> Parameters:
	gray: True
	patch_size: 44
	stride: 20
	max_number_patches: 128
	aug_times: 1
	trainset_dir: /content/FFDnet_clone/training/
	valset_dir: /content/FFDnet_clone/test/


> Training database
	Maximum number of patches set to 128
	file: /content/FFDnet_clone/training/test_001.png scale 1.0 # samples: 49
	file: /content/FFDnet_clone/training/test_001.png scale 0.9 # samples: 36
	file: /content/FFDnet_clone/training/test_001.png scale 0.8 # samples: 36
	file: /content/FFDnet_clone/training/test_001.png scale 0.7 # samples: 25

> Validation database
	file: /content/FFDnet_clone/test/test001.png
	file: /content/FFDnet_clone/test/test002.png
	file: /content/FFDnet_clone/test/test003.png
	file: /content/FFDnet_clone/test/test004.png
	file: /content/FFDnet_clone/test/test005.png
	file: /content/FFDnet_clone/test/test006.png
	file: /content/FFDnet_clone/test/test007.png
	file: /content/FFDnet_clone/test/test008.png
	file: /content/FFDnet_clone/test/test00

In [10]:
drive.mount('Drive')

Mounted at Drive


In [11]:
! mkdir /content/Drive/My\ Drive/ffdnetmodel/ #network weights are stored here

mkdir: cannot create directory ‘/content/Drive/My Drive/ffdnetmodel/’: File exists


In [9]:
! pip install tensorboardX

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.3 MB/s eta 0:00:00


In [12]:
! python /content/FFDnet_clone/train.py \
	--batch_size 128 \
	--epochs 80 \
	--noiseIntL 0 75 \
	--val_noiseL 25 \
  --log_dir /content/Drive/My\ Drive/ffdnetmodel/ \
  --gray


### Training FFDNet model ###
> Parameters:
	gray: True
	log_dir: /content/Drive/My Drive/ffdnetmodel/
	batch_size: 128
	epochs: 80
	resume_training: False
	milestone: [50, 60]
	lr: 0.001
	no_orthog: False
	save_every: 10
	save_every_epochs: 5
	noiseIntL: [0.0, 0.29411764705882354]
	val_noiseL: 0.09803921568627451


> Loading dataset ...
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
	# of training samples: 146

fatal: not a git repository (or any of the parent directories): .git
/content/FFDnet_clone/utils.py:33: FutureWarning: `nn.init.kaiming_normal` is now deprecated in favor of `

**NOTES**
* The training process can be monitored with TensorBoard as logs get saved
in the *log_dir* folder
* By default, models are trained for values of noise in [0, 75] (*--noiseIntL*
flag)
* By default, noise added at validation is set to 20 (*--val_noiseL* flag)
* A previous training can be resumed passing the *--resume_training* flag